# Basic Rag


In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Get OpenAI API key
openai_api_key = os.environ["OPENAI_API_KEY"]

print("API Key Loaded:", openai_api_key[:6] + "..." if openai_api_key else "No key found")


API Key Loaded: sk-pro...


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [12]:
loader = TextLoader("ms_dhoni.txt")
loaded_document = loader.load()

In [13]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [14]:
len(chunks_of_text)


56

In [15]:
embeddings = OpenAIEmbeddings()

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [16]:
vector_db

In [17]:
retriever = vector_db.as_retriever()


In [19]:
response = retriever.invoke("Who is thala in chennai?")
response

[Document(metadata={'source': 'ms_dhoni.txt'}, page_content="Beyond his international career, Dhoni made a massive impact in the Indian Premier League (IPL). As the captain of the \nChennai Super Kings (CSK), he led the team to multiple IPL titles and playoff appearances. His connection with CSK fans runs \ndeep, and he is adored as 'Thala' in Chennai, meaning 'leader' in Tamil. His calmness, humility, and loyalty towards the \nfranchise made him an IPL legend.\n\nOff the field, Dhoni is known for his simplicity and grounded personality. Despite his enormous success, he has remained \nhumble and approachable. He also has a passion for the Indian Army, holding the honorary rank of Lieutenant Colonel in the \nTerritorial Army. His love for bikes and cars is well known, and he has one of the most impressive collections among \ncricketers."),
 Document(metadata={'source': 'ms_dhoni.txt'}, page_content="Beyond his international career, Dhoni made a massive impact in the Indian Premier Leagu

In [20]:
len(response)


4

In [26]:
retriever = vector_db.as_retriever(search_kwargs={"k": 1})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x122267b50>, search_kwargs={'k': 1})

In [27]:
response = retriever.invoke("what is cks?")
response

[Document(metadata={'source': 'ms_dhoni.txt'}, page_content="Beyond his international career, Dhoni made a massive impact in the Indian Premier League (IPL). As the captain of the \nChennai Super Kings (CSK), he led the team to multiple IPL titles and playoff appearances. His connection with CSK fans runs \ndeep, and he is adored as 'Thala' in Chennai, meaning 'leader' in Tamil. His calmness, humility, and loyalty towards the \nfranchise made him an IPL legend.\n\nOff the field, Dhoni is known for his simplicity and grounded personality. Despite his enormous success, he has remained \nhumble and approachable. He also has a passion for the Indian Army, holding the honorary rank of Lieutenant Colonel in the \nTerritorial Army. His love for bikes and cars is well known, and he has one of the most impressive collections among \ncricketers.")]

# Simple use with LCEL

In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [29]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [30]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [38]:
response = chain.invoke("Which three ICC trophies did Dhoni win as captain, and why does this achievement set him apart from other cricket leaders?")
response

'Dhoni won the inaugural T20 World Cup in 2007, the 2011 ICC Cricket World Cup, and the 2013 ICC Champions Trophy as captain. This achievement sets him apart from other cricket leaders because he is the only captain to have won all three major ICC trophies.'

In [41]:
response = chain.invoke("what is dhoni salary?")
response

"Based on the context provided, it is not specified what Dhoni's salary is."

In [44]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

# loggin with mlflow

In [47]:
import os
import tempfile
from dotenv import load_dotenv, find_dotenv

# --- Load envs (expects OPENAI_API_KEY and optional MLFLOW_TRACKING_URI) ---
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "OPENAI_API_KEY missing in environment/.env"

import mlflow
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate

# ---------------- User-configurable knobs ----------------
EXPERIMENT_NAME = "Dhoni-RAG-with-MLflow"
LLM_MODEL = os.getenv("OPENAI_CHAT_MODEL", "gpt-4o-mini")  # set explicitly to log it
EMBED_MODEL = os.getenv("OPENAI_EMBED_MODEL", "text-embedding-3-small")
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
TOP_K = 2
DOC_PATH = "ms_dhoni.txt"

# ----------------- Setup MLflow -----------------
# If you already run `mlflow ui --host 127.0.0.1 --port 5000` this is fine.
# Otherwise set MLFLOW_TRACKING_URI in .env to your server.
mlflow.set_experiment(EXPERIMENT_NAME)

# ----------------- Build RAG pieces -----------------
loader = TextLoader(DOC_PATH)
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunks = splitter.split_documents(docs)

emb = OpenAIEmbeddings(model=EMBED_MODEL)
vdb = FAISS.from_documents(chunks, emb)
retriever = vdb.as_retriever(search_kwargs={"k": TOP_K})

prompt_tmpl = """Answer the question using ONLY the context.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{question}
"""
prompt = ChatPromptTemplate.from_template(prompt_tmpl)

# Keep chain simple so we get AIMessage back (with usage metadata)
llm = ChatOpenAI(model=LLM_MODEL, temperature=0)

def format_docs(ds):
    return "\n\n".join(d.page_content for d in ds)

def ask(question: str):
    context_docs = retriever.invoke(question)
    context_text = format_docs(context_docs)
    # produce AIMessage to capture token usage
    ai_msg = (prompt | llm).invoke({"context": context_text, "question": question})
    answer = ai_msg.content
    usage = ai_msg.response_metadata.get("token_usage", {})  # prompt_tokens, completion_tokens, total_tokens
    return {
        "question": question,
        "answer": answer,
        "context": context_text,
        "usage": usage,
    }

if __name__ == "__main__":
    with mlflow.start_run(run_name="rag_eval") as run:
        # ---- Log high-level params ----
        mlflow.log_params({
            "llm_model": LLM_MODEL,
            "embed_model": EMBED_MODEL,
            "chunk_size": CHUNK_SIZE,
            "chunk_overlap": CHUNK_OVERLAP,
            "retriever_k": TOP_K,
            "doc_path": DOC_PATH,
        })

        # ---- Log prompt template ----
        mlflow.log_text(prompt_tmpl, artifact_file="prompt_template.txt")

        # ---- Save and log FAISS index as artifact ----
        with tempfile.TemporaryDirectory() as tmpdir:
            idx_dir = os.path.join(tmpdir, "faiss_index")
            vdb.save_local(idx_dir)
            mlflow.log_artifacts(idx_dir, artifact_path="faiss_index")

        # ---- Ask a few questions and log results ----
        questions = [
            "Which three ICC trophies did Dhoni win as captain, and why is this unique?",
            "Who is called 'Thala' in Chennai?",
            "What is CSK?",
            "What is Dhoni's salary?",
        ]

        all_usage = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}

        for i, q in enumerate(questions, start=1):
            result = ask(q)

            # log each Q/A as text artifacts
            mlflow.log_text(result["question"], artifact_file=f"qa/{i:02d}_question.txt")
            mlflow.log_text(result["context"], artifact_file=f"qa/{i:02d}_retrieved_context.txt")
            mlflow.log_text(result["answer"], artifact_file=f"qa/{i:02d}_answer.txt")

            # log token metrics (if available)
            usage = result["usage"] or {}
            if "prompt_tokens" in usage:
                mlflow.log_metrics({
                    f"q{i}_prompt_tokens": usage.get("prompt_tokens", 0),
                    f"q{i}_completion_tokens": usage.get("completion_tokens", 0),
                    f"q{i}_total_tokens": usage.get("total_tokens", 0),
                })
                for k in all_usage:
                    all_usage[k] += usage.get(k, 0)

        # log aggregate token usage
        mlflow.log_metrics({
            "sum_prompt_tokens": all_usage["prompt_tokens"],
            "sum_completion_tokens": all_usage["completion_tokens"],
            "sum_total_tokens": all_usage["total_tokens"],
            "num_questions": len(questions),
        })

        print(f"MLflow run completed: {run.info.run_id}")
        print("Open MLflow UI at http://127.0.0.1:5000 and check the run artifacts/metrics/params.")


2025/08/23 21:18:45 INFO mlflow.tracking.fluent: Experiment with name 'Dhoni-RAG-with-MLflow' does not exist. Creating a new experiment.


MLflow run completed: 572a07f5fe30418b8d7689c7eff9404b
Open MLflow UI at http://127.0.0.1:5000 and check the run artifacts/metrics/params.


Name: mlflow
Version: 3.2.0
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: https://mlflow.org
Author: 
Author-email: 
License: Copyright 2018 Databricks, Inc.  All rights reserved.

                                Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the 